# The More The Merrier (Data Cleaning)

**Description:** This notebook is dedicated to the preprocessing and cleaning of raw data stored in *csv* files using the 
*Pandas* library, specifically focusing on three key datasets for this project.

- **Data:** Datasets to clean:
  - `2017_Entry_Exit.csv`
  - `2017_Average_Housing_Prices_in_London.csv`
  - ` LondonUnderground_Stations_Boroughs.csv `


In [93]:
# importing the necessary libraries
import pandas as pd
import numpy as np

In [94]:
def load_data(filepath):
    '''
    This function loads raw data from a csv file into a pandas dataframe and sets the primary key as the index column
    Args:
        filepath: the raw data's filepath in csv format
    Return:
        The loaded raw data into the pandas dataframe ready to be preprocessed
    '''
    df = pd.read_csv(filepath)
    return df 

### Data Wrangling 
**Description:** After loading the raw data into a dataframe with the function 'load_data', the next step is to wrangle the data. This process involves selecting the relevant columns for analysis, simplifying the column names and adjusting their data types. This process is done using a created class of objects that carries out the aforementioned tasks individually to a dataframe. The resulting dataframes are then ready to be cleaned.

In [132]:
class data_wrangler:
    '''
    This class does the necessary data wrangling such as selecting the relevant columns, setting each column's data type
    and renaming the columns.
    '''
    def __init__(self, df):
        self.df = df
    
    def get_data(self):
        return self.df
    
    def select_cols(self,cols):
        df = self.df[cols]
        cleaned_data = data_cleaner(df)
        return df

    def adjust_col_dtypes(self,col_dtypes):
        self.df = self.df.astype(col_dtypes)
        cleaned_data = data_cleaner(self.df)
        return self.df
    
    def rename_cols(self,rename):
        self.df.columns = rename
        cleaned_data = data_cleaner(self.df)
        return self.df

In [133]:
raw_df = load_data(r'C:\Users\pjxph\Documents\Data Science Projects\The More The Merrier\raw data\2017_Entry_Exit.csv')
# list of all london stations with it's each respective frequency of touch ins and touch outs.

In [134]:
df = data_cleaner(raw_df)
df = df.select_cols(['Station_ID', 'Station_Name','AnnualEntryExit_Mill'])
df = data_cleaner(df)
df = df.rename_cols(['id','name','frequency(millions)'])
df = data_cleaner(df)
df = df.adjust_col_dtypes({ 'id' : np.int64,
                           'name' : str,
                           'frequency(millions)' : np.float64
    
})
ldn_sta_freq = df
#cleaned_data.rename_cols(['id','name','z'])

In [135]:
# 2017_Average_hse_price
raw_df = load_data(r'C:\Users\pjxph\Documents/Data Science Projects/The More The Merrier/raw data/2017_Average_Housing_Prices_in_London.csv')

In [136]:
df = data_cleaner(raw_df)
df = df.select_cols(['Area_ID','Area_Name','average_hse_price'])
df = data_cleaner(df)
df = df.rename_cols(['id','name','avg_hse_price'])
df = data_cleaner(df)
df = df.adjust_col_dtypes({
    'id' : str,
    'name' : str,
    'avg_hse_price' : np.float64
})
ldn_hse_price = df 

In [137]:
raw_df = load_data(r'C:\Users\pjxph\Documents/Data Science Projects/The More The Merrier/raw data/LondonUnderground_Stations_Boroughs.csv')

In [138]:
raw_df.head()

,OBJECTID,NAME,NETWORK,Zone,area,Unnamed: 5,Unnamed: 6
0,1,Brent Cross,London Underground,3,Barnet,NaN,London Underground
1,2,Colindale,London Underground,4,Barnet,NaN,London Underground
2,3,Burnt Oak,London Underground,4,Barnet,NaN,London Underground
3,4,Edgware,London Underground,5,Barnet,NaN,London Underground
4,5,Mill Hill East,London Underground,4,Barnet,NaN,London Underground


In [139]:
df = data_cleaner(raw_df)
df = df.select_cols(['OBJECTID','NAME','area'])
df = data_cleaner(df)
df = df.rename_cols(['id','station_name','area_name'])
df = data_cleaner(df)
df = df.adjust_col_dtypes({
    'id' : np.int64,
    'station_name' : str,
    'area_name' : str
})
ldn_bor = df 

#### The three **wrangled** dataframes are:
* `ldn_sta_freq`
* `ldn_hse_price`
* `ldn_bor`

Time to clean!

### Data Cleaning
**Description:** After the data wrangling process, the wrangled data needs to be cleaned. The following processes were carried out:
- Ensure the consistency of the station name column in ldn_sta_freq and ldn_bor. 
  - The total number of stations should be the same.
  - Station names should be consistent

- Ensure the consistency of the area_name column in ldn_hse_price and ldn_bor.
  - The total number of london boroughs should be the same.
  - Borough names should be consistent.
- Ensure that each area name is consistent in ldn_bor

In [ ]:
class data_cleaner:
    '''
    This class does the necessary data cleaning such as removing duplicates and NaN values for the chosen column.
    '''
    def __init_(self, df):
        self.df = df
        
    def drop_dups(self):
        df = self.df.drop_duplicates()
        return df 
    
    def drop_na(self):
        df = self.df.drop(na)
        return df

In [156]:
print(len(ldn_sta_freq.index))
print(len(ldn_bor.index))

268
270


In [166]:
ldn_sta_freq.sort_values(by = 'name')
ldn_bor.sort_values(by = 'station_name')

,id,station_name,area_name
138,204,Acton Town,Ealing
82,84,Aldgate,City of London
183,249,Aldgate East,Tower Hamlets
12,13,Alperton,Brent
218,284,Amersham,Chiltern
...,...,...,...
98,100,Wimbledon Park,Merton
127,193,Wood Green,Haringey
268,334,Wood Lane,Hammersmith and Fulham
63,64,Woodford,Redbridge


In [201]:
x = [i for i in ldn_sta_freq.sort_values(by = 'name')['name']]

In [202]:
y = [j for j in ldn_bor.sort_values(by = 'station_name')['station_name']]

In [208]:
for item_x, item_y in zip(x, y):
    if item_x != item_y:
        print(item_x, item_y)

Park Royal Paddington
Parsons Green Paddington
Perivale Park Royal
Piccadilly Circus Parsons Green
Pimlico Perivale
Pinner Piccadilly Circus
Plaistow Pimlico
Preston Road Pinner
Putney Bridge Plaistow
Queen's Park Preston Road
Queensbury Putney Bridge
Queensway Queen's Park
Ravenscourt Park Queensbury
Rayners Lane Queensway
Redbridge Ravenscourt Park
Regent's Park Rayners Lane
Richmond Redbridge
Rickmansworth Regent's Park
Roding Valley Richmond
Royal Oak Rickmansworth
Ruislip Roding Valley
Ruislip Gardens Royal Oak
Ruislip Manor Ruislip
Russell Square Ruislip Gardens
Seven Sisters Ruislip Manor
Shepherd's Bush Russell Square
Shepherd's Bush Market Seven Sisters
Sloane Square Shepherd's Bush
Snaresbrook Shepherd's Bush Market
South Ealing Sloane Square
South Harrow Snaresbrook
South Kensington South Ealing
South Kenton South Harrow
South Ruislip South Kensington
South Wimbledon South Kenton
South Woodford South Ruislip
Southfields South Wimbledon
Southgate South Woodford
Southwark Sout